In [1]:
# import libraries

# 1. to handle the data
import pandas as pd
import numpy as np

# 2. To Viusalize the data
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from yellowbrick.cluster import KElbowVisualizer
from matplotlib.colors import ListedColormap

# 3. To preprocess the data
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
# 4. import Iterative imputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

# 5. Machine Learning
from sklearn.model_selection import train_test_split,GridSearchCV, cross_val_score

# 6. For Classification task.
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier,KNeighborsRegressor
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier,GradientBoostingClassifier,ExtraTreesClassifier,RandomForestRegressor
from xgboost import XGBClassifier


# 7. Metrics
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# 8. Ignore warnings
import warnings
warnings.filterwarnings('ignore')

from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, OrdinalEncoder

# improt ALl models.
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.naive_bayes import GaussianNB

#importing pipeline
from sklearn.pipeline import Pipeline

# import metrics
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, mean_absolute_error, mean_squared_error, r2_score


In [2]:
def read_data():
  train_df = pd.read_csv('train_data_with_ids.csv')
  test_df = pd.read_csv('test_data_with_ids.csv')
  return train_df, test_df


In [3]:
def drop(df):
  return df.dropna()

In [20]:
def imputeMean(df):
    ordinal_cols = ['slope', 'thal']
    nominal_cols = ['cp', 'restecg', 'sex']
    numerical_cols = ['oldpeak', 'thalch', 'chol', 'trestbps', 'age','ca']
    bool_cols = ['fbs', 'exang']

    # Fill numerical NAs with the column's mean value
    for col in numerical_cols:
        df[col].fillna(df[col].mean(), inplace=True)

    # Fill categorical NAs with the most frequent value (mode)
    for col in ordinal_cols + nominal_cols + bool_cols:
        df[col].fillna(df[col].mode()[0], inplace=True)

    return df

In [5]:
def imputeML(df):
    missing_data_cols = df.isnull().sum()[df.isnull().sum()>0].index.tolist()
    categorical_cols = ['thal', 'ca', 'slope', 'exang', 'restecg','fbs', 'cp', 'sex', 'num']
    bool_cols = ['fbs', 'exang']
    numerical_cols = ['oldpeak', 'thalch', 'chol', 'trestbps', 'age']
    passed_col = categorical_cols
    def impute_categorical_missing_data(passed_col):

        df_null = df[df[passed_col].isnull()]
        df_not_null = df[df[passed_col].notnull()]

        X = df_not_null.drop(passed_col, axis=1)
        y = df_not_null[passed_col]

        other_missing_cols = [col for col in missing_data_cols if col != passed_col]

        label_encoder = LabelEncoder()
        for col in X.columns:
            if X[col].dtype == 'object' or X[col].dtype == 'category':
                X[col] = label_encoder.fit_transform(X[col])

        if passed_col in bool_cols:
            y = label_encoder.fit_transform(y)

        iterative_imputer = IterativeImputer(estimator=RandomForestRegressor(random_state=42), add_indicator=True)

        for col in other_missing_cols:
            if X[col].isnull().sum() > 0:
                col_with_missing_values = X[col].values.reshape(-1, 1)
                imputed_values = iterative_imputer.fit_transform(col_with_missing_values)
                X[col] = imputed_values[:, 0]
            else:
                pass

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        rf_classifier = RandomForestClassifier()

        rf_classifier.fit(X_train, y_train)

        y_pred = rf_classifier.predict(X_test)

        acc_score = accuracy_score(y_test, y_pred)

        print("The feature '"+ passed_col+ "' has been imputed with", round((acc_score * 100), 2), "accuracy\n")

        X = df_null.drop(passed_col, axis=1)

        for col in X.columns:
            if X[col].dtype == 'object' or X[col].dtype == 'category':
                X[col] = label_encoder.fit_transform(X[col])

        for col in other_missing_cols:
            if X[col].isnull().sum() > 0:
                col_with_missing_values = X[col].values.reshape(-1, 1)
                imputed_values = iterative_imputer.fit_transform(col_with_missing_values)
                X[col] = imputed_values[:, 0]
            else:
                pass

        if len(df_null) > 0:
            df_null[passed_col] = rf_classifier.predict(X)
            if passed_col in bool_cols:
                df_null[passed_col] = df_null[passed_col].map({0: False, 1: True})
            else:
                pass
        else:
            pass

        df_combined = pd.concat([df_not_null, df_null])

        return df_combined[passed_col]

    def impute_continuous_missing_data(passed_col):

        df_null = df[df[passed_col].isnull()]
        df_not_null = df[df[passed_col].notnull()]

        X = df_not_null.drop(passed_col, axis=1)
        y = df_not_null[passed_col]

        other_missing_cols = [col for col in missing_data_cols if col != passed_col]

        label_encoder = LabelEncoder()

        for col in X.columns:
            if X[col].dtype == 'object' or X[col].dtype == 'category':
                X[col] = label_encoder.fit_transform(X[col])

        iterative_imputer = IterativeImputer(estimator=RandomForestRegressor(random_state=42), add_indicator=True)

        for col in other_missing_cols:
            if X[col].isnull().sum() > 0:
                col_with_missing_values = X[col].values.reshape(-1, 1)
                imputed_values = iterative_imputer.fit_transform(col_with_missing_values)
                X[col] = imputed_values[:, 0]
            else:
                pass

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        rf_regressor = RandomForestRegressor()

        rf_regressor.fit(X_train, y_train)

        y_pred = rf_regressor.predict(X_test)

        print("MAE =", mean_absolute_error(y_test, y_pred), "\n")
        print("RMSE =", mean_squared_error(y_test, y_pred, squared=False), "\n")
        print("R2 =", r2_score(y_test, y_pred), "\n")

        X = df_null.drop(passed_col, axis=1)

        for col in X.columns:
            if X[col].dtype == 'object' or X[col].dtype == 'category':
                X[col] = label_encoder.fit_transform(X[col])

        for col in other_missing_cols:
            if X[col].isnull().sum() > 0:
                col_with_missing_values = X[col].values.reshape(-1, 1)
                imputed_values = iterative_imputer.fit_transform(col_with_missing_values)
                X[col] = imputed_values[:, 0]
            else:
                pass

        if len(df_null) > 0:
            df_null[passed_col] = rf_regressor.predict(X)
        else:
            pass

        df_combined = pd.concat([df_not_null, df_null])

        return df_combined[passed_col]

    import warnings
    warnings.filterwarnings('ignore')

    # impute missing values using our functions
    for col in missing_data_cols:
        print("Missing Values", col, ":", str(round((df[col].isnull().sum() / len(df)) * 100, 2))+"%")
        if col in categorical_cols:
            df[col] = impute_categorical_missing_data(col)
        elif col in numerical_cols:
            df[col] = impute_continuous_missing_data(col)
    return df

In [21]:
def encodeDf(imputeMethod):

  train_df, test_df = read_data()
  # Concatenate train and test DataFrames
  train_df['ifTrain'] = 1
  test_df['ifTrain'] = 0
  df_concatenated = pd.concat([train_df, test_df], ignore_index=True)

  df_concatenated = imputeMethod(df_concatenated)

  # Define column categories
  ordinal_cols = ['slope', 'thal']
  nominal_cols = ['cp', 'restecg', 'sex']
  bool_cols = ['exang', 'fbs']
  numerical_cols = ['oldpeak', 'thalch', 'chol', 'trestbps', 'age']

  # Initialize encoders and scaler
  minmax_scaler = MinMaxScaler()
  onehot_encoder = OneHotEncoder(drop='first', sparse=False)  # drop first to avoid dummy trap
  ordinal_encoder = OrdinalEncoder()

  # Scale numerical columns
  df_concatenated[numerical_cols] = minmax_scaler.fit_transform(df_concatenated[numerical_cols])

  # Encode ordinal columns
  df_concatenated[ordinal_cols] = ordinal_encoder.fit_transform(df_concatenated[ordinal_cols])

  # One-hot encode nominal columns and get a DataFrame with new column names
  onehot_encoded = onehot_encoder.fit_transform(df_concatenated[nominal_cols])
  df_nominal_encoded = pd.DataFrame(onehot_encoded, columns=onehot_encoder.get_feature_names_out())

  # Concatenate the one-hot encoded nominal DataFrame with the original df
  # Make sure to reset the index before concatenation to align the rows correctly
  df_concatenated = df_concatenated.reset_index(drop=True)
  df_nominal_encoded = df_nominal_encoded.reset_index(drop=True)
  df_encoded = pd.concat([df_concatenated, df_nominal_encoded], axis=1)

  # Now drop the original nominal columns as they have been one-hot encoded
  df_encoded = df_encoded.drop(columns=nominal_cols)

  # Ensure the 'id' column is dropped if it's not needed
  df_encoded = df_encoded.drop(columns=['id'])
  df_encoded = df_encoded.drop('dataset', axis=1)
  df_encoded['fbs'] = df_encoded['fbs'].astype(float)
  df_encoded['exang'] = df_encoded['exang'].astype(float)


  # Split the encoded DataFrame back into train and test sets based on 'ifTrain' column
  train_set = df_encoded[df_encoded['ifTrain'] == 1].drop(columns=['ifTrain'])
  test_set = df_encoded[df_encoded['ifTrain'] == 0].drop(columns=['ifTrain'])

  # Optionally, you can also split your train data into train and validation sets if needed
  # X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

  y_train = train_set['num']
  X_train = train_set.drop(columns = ['num'])
  y_test = test_set['num']
  X_test = test_set.drop(columns = ['num'])

  return X_train, X_test, y_train, y_test

In [11]:
X_train, X_test, y_train, y_test = encodeDf(imputeMethod = imputeML)

Missing Values trestbps : 6.41%
MAE = 15.096242774566475 

RMSE = 20.989818162003893 

R2 = -0.003806314903271879 

Missing Values chol : 3.26%
MAE = 43.64258426966292 

RMSE = 64.61237512350961 

R2 = 0.6489413238162391 

Missing Values fbs : 9.78%
The feature 'fbs' has been imputed with 83.73 accuracy

Missing Values restecg : 0.22%
The feature 'restecg' has been imputed with 63.59 accuracy

Missing Values thalch : 5.98%
MAE = 16.526416184971097 

RMSE = 20.6979112765968 

R2 = 0.3881620621652334 

Missing Values exang : 5.98%
The feature 'exang' has been imputed with 80.92 accuracy

Missing Values oldpeak : 6.74%
MAE = 0.5180290697674419 

RMSE = 0.7707210946989537 

R2 = 0.5048796729931088 

Missing Values slope : 33.59%
The feature 'slope' has been imputed with 75.61 accuracy

Missing Values ca : 66.41%
The feature 'ca' has been imputed with 58.06 accuracy

Missing Values thal : 52.83%
The feature 'thal' has been imputed with 59.77 accuracy



In [12]:
X_test

,age,trestbps,chol,fbs,thalch,exang,oldpeak,slope,ca,thal,cp_atypical angina,cp_non-anginal,cp_typical angina,restecg_normal,restecg_st-t abnormality,sex_Male
860,0.612245,0.570,0.527363,0.0,0.563380,0.0,0.795455,0.0,3.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
861,0.285714,0.650,0.298507,0.0,0.633803,0.0,0.295455,2.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0
862,0.591837,0.660,0.343284,0.0,0.760563,1.0,0.295455,2.0,0.0,2.0,0.0,0.0,0.0,1.0,0.0,1.0
863,0.510204,0.700,0.336650,1.0,0.669014,1.0,0.647727,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0
864,0.469388,0.700,0.510779,0.0,0.577465,0.0,0.465909,2.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
865,0.551020,0.640,0.339967,0.0,0.492958,1.0,0.522727,1.0,1.0,2.0,0.0,0.0,0.0,0.0,1.0,0.0
866,0.714286,0.675,0.417910,0.0,0.788732,0.0,0.295455,2.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
867,0.673469,0.725,0.509121,0.0,0.605634,1.0,0.409091,1.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0
868,0.571429,0.600,0.391376,0.0,0.830986,0.0,0.386364,2.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0
869,0.591837,0.760,0.454395,0.0,0.197183,1.0,0.431818,1.0,1.0,2.0,0.0,0.0,0.0,1.0,0.0,1.0


In [13]:
import numpy as np
import pandas as pd
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

# 定义模型和它们的参数网格
model_params = {
    'Logistic Regression': {
        'model': LogisticRegression(),
        'params': {
            'C': [0.1, 1, 10, 100],
            'solver': ['liblinear', 'lbfgs','sag','saga']
        }
    },
    'Decision Tree': {
        'model': DecisionTreeClassifier(),
        'params': {
            'max_depth': [None, 10, 20, 30],
            'min_samples_split': [2, 5, 10]
        }
    },
    'Bayesian Classifier': {
        'model': GaussianNB(),
        'params': {
            'var_smoothing': [1e-9, 1e-8, 1e-7]
        }
    }
}

# 存储最优模型和结果
best_models = {}
results = []

# 对每种模型执行网格搜索
for model_name, mp in model_params.items():
    clf = GridSearchCV(mp['model'], mp['params'], cv=5, return_train_score=False)
    clf.fit(X_train, y_train)
    best_models[model_name] = clf.best_estimator_
    y_pred = clf.best_estimator_.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred)
    results.append({
        'model': model_name,
        'best_score': clf.best_score_,
        'best_params': clf.best_params_,
        'test_accuracy': accuracy,
        'classification_report': report
    })

# 打印每种模型的性能和参数
for result in results:
    print(f"Model: {result['model']}")
    print("Best Score (CV):", result['best_score'])
    print("Best Parameters:", result['best_params'])
    print("Test Accuracy:", result['test_accuracy'])
    print("Classification Report:\n", result['classification_report'])
    print("-" * 30)

# 比较不同模型
best_model = max(results, key=lambda x: x['test_accuracy'])
print(f"Overall Best Model: {best_model['model']} with Test Accuracy: {best_model['test_accuracy']}")


Model: Logistic Regression
Best Score (CV): 0.8162790697674419
Best Parameters: {'C': 0.1, 'solver': 'liblinear'}
Test Accuracy: 0.8333333333333334
Classification Report:
               precision    recall  f1-score   support

           0       0.88      0.83      0.85        35
           1       0.78      0.84      0.81        25

    accuracy                           0.83        60
   macro avg       0.83      0.83      0.83        60
weighted avg       0.84      0.83      0.83        60

------------------------------
Model: Decision Tree
Best Score (CV): 0.7325581395348838
Best Parameters: {'max_depth': 10, 'min_samples_split': 2}
Test Accuracy: 0.8166666666666667
Classification Report:
               precision    recall  f1-score   support

           0       0.83      0.86      0.85        35
           1       0.79      0.76      0.78        25

    accuracy                           0.82        60
   macro avg       0.81      0.81      0.81        60
weighted avg       0.82 

In [22]:
X_train, X_test, y_train, y_test = encodeDf(imputeMethod = imputeMean)

In [23]:
(X_train.isnull().sum()/ len(X_train)* 100).sort_values(ascending=False)

age                         0.0
trestbps                    0.0
chol                        0.0
fbs                         0.0
thalch                      0.0
exang                       0.0
oldpeak                     0.0
slope                       0.0
ca                          0.0
thal                        0.0
cp_atypical angina          0.0
cp_non-anginal              0.0
cp_typical angina           0.0
restecg_normal              0.0
restecg_st-t abnormality    0.0
sex_Male                    0.0
dtype: float64

In [24]:
import numpy as np
import pandas as pd
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

# 定义模型和它们的参数网格
model_params = {
    'Logistic Regression': {
        'model': LogisticRegression(),
        'params': {
            'C': [0.1, 1, 10, 100],
            'solver': ['liblinear', 'lbfgs','sag','saga']
        }
    },
    'Decision Tree': {
        'model': DecisionTreeClassifier(),
        'params': {
            'max_depth': [None, 10, 20, 30],
            'min_samples_split': [2, 5, 10]
        }
    },
    'Bayesian Classifier': {
        'model': GaussianNB(),
        'params': {
            'var_smoothing': [1e-9, 1e-8, 1e-7]
        }
    }
}

# 存储最优模型和结果
best_models = {}
results = []

# 对每种模型执行网格搜索
for model_name, mp in model_params.items():
    clf = GridSearchCV(mp['model'], mp['params'], cv=5, return_train_score=False)
    clf.fit(X_train, y_train)
    best_models[model_name] = clf.best_estimator_
    y_pred = clf.best_estimator_.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred)
    results.append({
        'model': model_name,
        'best_score': clf.best_score_,
        'best_params': clf.best_params_,
        'test_accuracy': accuracy,
        'classification_report': report
    })

# 打印每种模型的性能和参数
for result in results:
    print(f"Model: {result['model']}")
    print("Best Score (CV):", result['best_score'])
    print("Best Parameters:", result['best_params'])
    print("Test Accuracy:", result['test_accuracy'])
    print("Classification Report:\n", result['classification_report'])
    print("-" * 30)

# 比较不同模型
best_model = max(results, key=lambda x: x['test_accuracy'])
print(f"Overall Best Model: {best_model['model']} with Test Accuracy: {best_model['test_accuracy']}")


Model: Logistic Regression
Best Score (CV): 0.7848837209302325
Best Parameters: {'C': 0.1, 'solver': 'liblinear'}
Test Accuracy: 0.8166666666666667
Classification Report:
               precision    recall  f1-score   support

           0       0.83      0.86      0.85        35
           1       0.79      0.76      0.78        25

    accuracy                           0.82        60
   macro avg       0.81      0.81      0.81        60
weighted avg       0.82      0.82      0.82        60

------------------------------
Model: Decision Tree
Best Score (CV): 0.6709302325581394
Best Parameters: {'max_depth': 10, 'min_samples_split': 2}
Test Accuracy: 0.7
Classification Report:
               precision    recall  f1-score   support

           0       0.74      0.74      0.74        35
           1       0.64      0.64      0.64        25

    accuracy                           0.70        60
   macro avg       0.69      0.69      0.69        60
weighted avg       0.70      0.70      

In [18]:
X_train, X_test, y_train, y_test = encodeDf(imputeMethod = drop)

In [19]:
import numpy as np
import pandas as pd
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

# 定义模型和它们的参数网格
model_params = {
    'Logistic Regression': {
        'model': LogisticRegression(),
        'params': {
            'C': [0.1, 1, 10, 100],
            'solver': ['liblinear', 'lbfgs','sag','saga']
        }
    },
    'Decision Tree': {
        'model': DecisionTreeClassifier(),
        'params': {
            'max_depth': [None, 10, 20, 30],
            'min_samples_split': [2, 5, 10]
        }
    },
    'Bayesian Classifier': {
        'model': GaussianNB(),
        'params': {
            'var_smoothing': [1e-9, 1e-8, 1e-7]
        }
    }
}

# 存储最优模型和结果
best_models = {}
results = []

# 对每种模型执行网格搜索
for model_name, mp in model_params.items():
    clf = GridSearchCV(mp['model'], mp['params'], cv=5, return_train_score=False)
    clf.fit(X_train, y_train)
    best_models[model_name] = clf.best_estimator_
    y_pred = clf.best_estimator_.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred)
    results.append({
        'model': model_name,
        'best_score': clf.best_score_,
        'best_params': clf.best_params_,
        'test_accuracy': accuracy,
        'classification_report': report
    })

# 打印每种模型的性能和参数
for result in results:
    print(f"Model: {result['model']}")
    print("Best Score (CV):", result['best_score'])
    print("Best Parameters:", result['best_params'])
    print("Test Accuracy:", result['test_accuracy'])
    print("Classification Report:\n", result['classification_report'])
    print("-" * 30)

# 比较不同模型
best_model = max(results, key=lambda x: x['test_accuracy'])
print(f"Overall Best Model: {best_model['model']} with Test Accuracy: {best_model['test_accuracy']}")


Model: Logistic Regression
Best Score (CV): 0.8324468085106383
Best Parameters: {'C': 0.1, 'solver': 'lbfgs'}
Test Accuracy: 0.8833333333333333
Classification Report:
               precision    recall  f1-score   support

           0       0.87      0.94      0.90        35
           1       0.91      0.80      0.85        25

    accuracy                           0.88        60
   macro avg       0.89      0.87      0.88        60
weighted avg       0.89      0.88      0.88        60

------------------------------
Model: Decision Tree
Best Score (CV): 0.7155141843971631
Best Parameters: {'max_depth': 20, 'min_samples_split': 10}
Test Accuracy: 0.8
Classification Report:
               precision    recall  f1-score   support

           0       0.87      0.77      0.82        35
           1       0.72      0.84      0.78        25

    accuracy                           0.80        60
   macro avg       0.80      0.81      0.80        60
weighted avg       0.81      0.80      0.8